In [1]:
from pyspark.sql import *
from pyspark.ml.feature import *
from pyspark.ml.regression import *

In [2]:
spark = SparkSession.builder.appName("Amith").getOrCreate()
training = spark.read.csv("test1.csv",header=True,inferSchema=True)

In [42]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [43]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [44]:
featureassembler = VectorAssembler(inputCols=["age","Experience"],outputCol="transformed")
output = featureassembler.transform(training)

In [50]:
df = output.select(["transformed","Salary"])
df.show()

+-----------+------+
|transformed|Salary|
+-----------+------+
|[31.0,10.0]| 30000|
| [30.0,8.0]| 25000|
| [29.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
+-----------+------+



In [55]:
train_data,test_data = df.randomSplit([0.75,0.25])
train_data.show()

+-----------+------+
|transformed|Salary|
+-----------+------+
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
| [24.0,3.0]| 20000|
|[31.0,10.0]| 30000|
+-----------+------+



In [57]:
regressor = LinearRegression(featuresCol="transformed",labelCol="Salary")
regressor = regressor.fit(test_data)

In [59]:
print(regressor.coefficients)

[2500.0000000003815,625.0000000000011]


In [60]:
print(regressor.intercept)

-55000.00000001126


In [61]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+-----------+------+------------------+
|transformed|Salary|        prediction|
+-----------+------+------------------+
| [29.0,4.0]| 20000|19999.999999999796|
| [30.0,8.0]| 25000|25000.000000000204|
+-----------+------+------------------+



In [63]:
pred_results.meanAbsoluteError

2.0372681319713593e-10